In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 29.9 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import re
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from transformers import AutoTokenizer, AutoProcessor, AutoModelForCTC, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import numpy as np
import torch
from torch import nn
import librosa
import gradio as gr
import easyocr

In [4]:
from google.colab import drive
# 2. THIẾT LẬP ĐƯỜNG DẪN VÀ TẢI TOKENIZER
print("\n--- 2. Kết nối Drive và Tải Tokenizer ---")
drive.mount('/content/drive')
# ĐƯỜNG DẪN ĐẾN THƯ MỤC CHỨA 3 FILE CSV
base_dir = '/content/drive/My Drive/DỰ ÁN CNTT/ViCTSD'

# Tên các file
train_file = 'ViCTSD_train.csv'
valid_file = 'ViCTSD_valid.csv'
test_file = 'ViCTSD_test.csv'

PHOBERT_MODEL = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(PHOBERT_MODEL)
print(f"✅ PhoBERT Tokenizer ({PHOBERT_MODEL}) đã được tải.")

# 3. ĐỊNH NGHĨA HÀM TIỀN XỬ LÝ VÀ TOKEN HÓA
def basic_cleanup(text):
    """Làm sạch cơ bản (thay thế Tách từ VnCoreNLP)."""
    text = str(text)
    text = re.sub(r'[^\w\s]', ' ', text).lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def load_and_tokenize_data_direct(filename, base_dir, tokenizer):
    """Đọc file, làm sạch và Token hóa trực tiếp."""
    file_path = os.path.join(base_dir, filename)
    print(f"\n--- Xử lý file: {filename} ---")

    if not os.path.exists(file_path):
        print(f"❌ LỖI: Không tìm thấy file tại {file_path}. Bỏ qua.")
        return None

    df = pd.read_csv(file_path)
    df_clean = df[['Comment', 'Toxicity']].copy()
    df_clean.rename(columns={'Comment': 'text', 'Toxicity': 'label'}, inplace=True)

    # Làm sạch văn bản
    df_clean['text'] = df_clean['text'].apply(basic_cleanup)

    # Chuyển đổi sang HuggingFace Dataset
    dataset = Dataset.from_pandas(df_clean[['text', 'label']])

    # Áp dụng Tokenization TRỰC TIẾP
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_dataset = tokenized_dataset.remove_columns(["text"]) # Xóa cột văn bản thô

    print(f"✅ Hoàn thành. Kích thước dataset: {len(tokenized_dataset)}")
    return tokenized_dataset

# 4. THỰC HIỆN TẢI VÀ TOKEN HÓA
train_tokenized = load_and_tokenize_data_direct(train_file, base_dir, tokenizer)
val_tokenized = load_and_tokenize_data_direct(valid_file, base_dir, tokenizer)
test_tokenized = load_and_tokenize_data_direct(test_file, base_dir, tokenizer)


--- 2. Kết nối Drive và Tải Tokenizer ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

...

✅ Hoàn thành. Kích thước dataset: 7000

--- Xử lý file: ViCTSD_valid.csv ---


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

✅ Hoàn thành. Kích thước dataset: 2000

--- Xử lý file: ViCTSD_test.csv ---


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

✅ Hoàn thành. Kích thước dataset: 1000


In [5]:
# --- HÀM TÔ MÀU VÀ DANH SÁCH TỪ ĐỘC HẠI ---
# Danh sách mẫu: Cần được mở rộng dựa trên phân tích dữ liệu ViCTSD
TOXIC_WORDS = set([
    'ngu dốt', 'cút', 'chó', 'điên', 'óc chó', 'rác rưởi', 'khùng',
    'mày', 'tao', 'đồ ngu', 'mất dạy', 'chết đi', 'thằng', 'con'
])

def highlight_toxic_words(text):
    """Tô màu đỏ các từ/cụm từ độc hại trong văn bản và trả về chuỗi HTML."""
    if not text:
        return ""

    highlighted_text = text

    # Sắp xếp từ độc hại theo độ dài giảm dần để cụm từ dài được ưu tiên
    sorted_words = sorted(list(TOXIC_WORDS), key=len, reverse=True)

    for word in sorted_words:
        # Dùng re.escape để xử lý các ký tự đặc biệt trong từ độc hại
        # Dùng r'\b(' + word + r')\b' để đảm bảo tìm thấy toàn bộ từ (word boundaries)
        pattern = r'\b(' + re.escape(word) + r')\b'

        # Tô màu đỏ đậm bằng thẻ HTML
        highlighted_text = re.sub(
            pattern,
            r'<span style="color:red; font-weight:bold;">\1</span>',
            highlighted_text,
            flags=re.IGNORECASE
        )

    return highlighted_text

# --- HÀM OCR VÀ PHÂN LOẠI HÌNH ẢNH ---
def ocr_and_predict(image_file, tokenizer, model, reader):
    """Thực hiện OCR trên ảnh và phân loại văn bản trích xuất."""
    ocr_text = ""
    is_toxic = 0
    confidence = 0.5

    if image_file is not None and reader is not None:
        try:
            # 1. Thực hiện OCR
            results = reader.readtext(image_file)

            # 2. Nối tất cả văn bản trích xuất thành một chuỗi
            if results:
                # res[1] là văn bản trích xuất
                ocr_text = " ".join([res[1] for res in results])

            # 3. Phân loại HSD
            if ocr_text:
                # classify_text_toxicity cần được định nghĩa trước
                is_toxic, confidence = classify_text_toxicity(ocr_text, tokenizer, model)

        except Exception as e:
            ocr_text = f"❌ LỖI OCR: {e}"
            is_toxic = 0
            confidence = 0.5

    return ocr_text, is_toxic, confidence

In [ ]:
!pip install --upgrade transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 140.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0


In [6]:
# ===================================================================
# SECTION PHOBERT (HUẤN LUYỆN MÔ HÌNH VĂN BẢN)
#
# Mục tiêu: Huấn luyện PhoBERT trên 3 file CSV để phân loại
# văn bản độc hại, có xử lý mất cân bằng dữ liệu (weighted loss).
# ===================================================================

# --- 0. IMPORT CÁC THƯ VIỆN CẦN THIẾT ---
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np
import torch
from torch import nn

# --- KIỂM TRA ĐIỀU KIỆN TIÊN QUYẾT ---
# Đảm bảo 'train_tokenized' (từ ô code Tải dữ liệu) đã được tạo
try:
    if 'train_tokenized' not in locals() or 'val_tokenized' not in locals() or 'tokenizer' not in locals():
        print("❌ LỖI: Biến 'train_tokenized', 'val_tokenized', hoặc 'tokenizer' không tồn tại.")
        print("Vui lòng chạy lại ô code TẢI DỮ LIỆU (Data Loading) ở trên trước.")
        raise NameError("Dữ liệu chưa được tải")
except NameError as e:
    # Dừng thực thi nếu thiếu dữ liệu
    raise e

# --- 1. TÍNH TOÁN TRỌNG SỐ LỚP ---
print("\n--- 1. Bắt đầu tính toán trọng số lớp ---")
# Đảm bảo sử dụng GPU nếu có
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Thiết bị sử dụng: {device}")

# Tải nhãn từ tập huấn luyện để tính toán trọng số
train_labels = np.array(train_tokenized['label'])
num_clean = np.sum(train_labels == 0) # Số lượng mẫu Sạch (0)
num_toxic = np.sum(train_labels == 1) # Số lượng mẫu Độc hại (1)
total_samples = len(train_labels)

# Tính trọng số nghịch đảo (Inverse Frequency)
# Công thức: Tổng mẫu / (2 * Số mẫu của lớp đó)
weight_0 = total_samples / (2.0 * num_clean)
weight_1 = total_samples / (2.0 * num_toxic)

# Chuyển trọng số thành Tensor và đưa lên GPU
class_weights = torch.tensor([weight_0, weight_1], dtype=torch.float).to(device)
print(f"Tổng mẫu Train: {total_samples}. Sạch: {num_clean}, Độc hại: {num_toxic}")
print(f"Trọng số lớp đã tính: Sạch (0) = {weight_0:.4f}, Độc hại (1) = {weight_1:.4f}")

# --- 2. ĐỊNH NGHĨA MÔ HÌNH PHOBERT CÓ TRỌNG SỐ ---
print("\n--- 2. Định nghĩa mô hình PhoBERT có trọng số ---")
PHOBERT_MODEL = "vinai/phobert-base"

class PhoBERT_Weighted(AutoModelForSequenceClassification):
    """Mô hình kế thừa để sử dụng Cross-Entropy Loss có trọng số."""
    def __init__(self, config):
        super().__init__(config)
        # Định nghĩa hàm mất mát (CrossEntropyLoss) với trọng số TÍNH ĐƯỢC
        self.loss_fct = nn.CrossEntropyLoss(weight=class_weights)

    def forward(self, input_ids=None, labels=None, **kwargs):
        # Chạy forward pass gốc của mô hình
        outputs = super().forward(input_ids=input_ids, labels=labels, **kwargs)

        if labels is None:
            return outputs

        # Tính toán lại loss bằng hàm mất mát có trọng số
        logits = outputs.logits
        # Đảm bảo labels có kiểu long (int64)
        loss = self.loss_fct(logits, labels.long())

        outputs.loss = loss
        return outputs

# Tải mô hình PhoBERT mới, sử dụng lớp Weighted
model_weighted = PhoBERT_Weighted.from_pretrained(PHOBERT_MODEL, num_labels=2).to(device)
print(f"✅ Tải thành công mô hình {PHOBERT_MODEL} với hàm loss tùy chỉnh.")

# --- 3. ĐỊNH NGHĨA HÀM ĐÁNH GIÁ VÀ THAM SỐ HUẤN LUYỆN ---
print("\n--- 3. Định nghĩa hàm đánh giá và tham số huấn luyện ---")

# Định nghĩa hàm đánh giá hiệu suất (compute_metrics)
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids

    f1 = f1_score(labels, preds, average='binary')
    precision = precision_score(labels, preds, average='binary')
    recall = recall_score(labels, preds, average='binary')
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'f1_score': f1,
        'precision': precision,
        'recall': recall,
    }

# Định nghĩa Tham số Huấn luyện
training_args = TrainingArguments(
    output_dir="./phobert_toxic_weighted_results",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./phobert_toxic_weighted_logs',
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True if device.type == 'cuda' else False,
    report_to="none"
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model_weighted, # SỬ DỤNG MODEL MỚI CÓ TRỌNG SỐ
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer, # Sử dụng tokenizer đã tải từ các bước trước
    compute_metrics=compute_metrics,
)

# --- 4. BẮT ĐẦU HUẤN LUYỆN ---
print("\n--- 4. BẮT ĐẦU TÁI HUẤN LUYỆN MÔ HÌNH PHOBERT CÓ TRỌNG SỐ ---")
trainer.train()

# --- 5. LƯU MÔ HÌNH ---
print("\n--- 5. Lưu mô hình tốt nhất ---")
# Lưu mô hình cuối cùng (tốt nhất)
trainer.save_model("./phobert_toxic_weighted_final_model")
tokenizer.save_pretrained("./phobert_toxic_weighted_final_model")

print("\n--- TÁI HUẤN LUYỆN HOÀN TẤT ---")
print("✅ Mô hình đã được lưu vào thư mục: ./phobert_toxic_weighted_final_model")


--- 1. Bắt đầu tính toán trọng số lớp ---
Thiết bị sử dụng: cuda
Tổng mẫu Train: 7000. Sạch: 6241, Độc hại: 759
Trọng số lớp đã tính: Sạch (0) = 0.5608, Độc hại (1) = 4.6113

--- 2. Định nghĩa mô hình PhoBERT có trọng số ---


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Tải thành công mô hình vinai/phobert-base với hàm loss tùy chỉnh.

--- 3. Định nghĩa hàm đánh giá và tham số huấn luyện ---


/tmp/ipython-input-2031399505.py:116: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



--- 4. BẮT ĐẦU TÁI HUẤN LUYỆN MÔ HÌNH PHOBERT CÓ TRỌNG SỐ ---


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score,Precision,Recall
1,0.302400,0.273191,0.900000,0.481865,0.603896,0.400862
2,0.269200,0.293317,0.902500,0.368932,0.740260,0.245690
3,0.205900,0.371904,0.890000,0.481132,0.531250,0.439655


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score,Precision,Recall
1,0.302400,0.273191,0.900000,0.481865,0.603896,0.400862
2,0.269200,0.293317,0.902500,0.368932,0.740260,0.245690
3,0.205900,0.371904,0.890000,0.481132,0.531250,0.439655
4,0.161100,0.405793,0.895500,0.459948,0.574194,0.383621
5,0.082800,0.528769,0.900500,0.451791,0.625954,0.353448
6,0.037500,0.620438,0.904500,0.473829,0.656489,0.370690
7,0.027800,0.626899,0.903000,0.492147,0.626667,0.405172
8,0.023700,0.767865,0.901000,0.484375,0.611842,0.400862
9,0.006200,0.805516,0.899000,0.492462,0.590361,0.422414
10,0.007200,0.823907,0.901000,0.476190,0.616438,0.387931



--- 5. Lưu mô hình tốt nhất ---

--- TÁI HUẤN LUYỆN HOÀN TẤT ---
✅ Mô hình đã được lưu vào thư mục: ./phobert_toxic_weighted_final_model


In [9]:
print("\n--- Bắt đầu Đánh giá trên Tập Test (Test Set) ---")

# Thực hiện đánh giá
test_results = trainer.evaluate(test_tokenized)

# Hiển thị kết quả
print("\n🔥 KẾT QUẢ ĐÁNH GIÁ CUỐI CÙNG (TEST SET):")
for key, value in test_results.items():
    print(f"  {key}: {value:.4f}")

# (Tùy chọn) Dự đoán và In Ma trận Nhầm lẫn
predictions = trainer.predict(test_tokenized)
preds = np.argmax(predictions.predictions, axis=1)
labels = test_tokenized['label']

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels, preds)
print("\nMa trận Nhầm lẫn (Confusion Matrix):")
print(cm)


--- Bắt đầu Đánh giá trên Tập Test (Test Set) ---



🔥 KẾT QUẢ ĐÁNH GIÁ CUỐI CÙNG (TEST SET):
  eval_loss: 0.2896
  eval_accuracy: 0.8860
  eval_f1_score: 0.4519
  eval_precision: 0.4796
  eval_recall: 0.4273
  eval_runtime: 4.3664
  eval_samples_per_second: 229.0230
  eval_steps_per_second: 14.4280
  epoch: 10.0000

Ma trận Nhầm lẫn (Confusion Matrix):
[[839  51]
 [ 63  47]]


In [19]:
# --- TẢI BLACKLIST TỪ FILE CSV ---

# (!!!) Thay đổi đường dẫn này cho đúng với Google Drive của bạn
BLACKLIST_PATH = "/content/drive/My Drive/DỰ ÁN CNTT/ViCTSD/blacklist.csv"

def load_blacklist(path):
    """Đọc file CSV và trả về một set các từ độc hại."""
    try:
        df = pd.read_csv(path)
        # Lọc bỏ các dòng comment (bắt đầu bằng #)
        df_clean = df[~df['term'].str.startswith('#', na=False)]

        # Chuyển tất cả về chữ thường và loại bỏ khoảng trắng
        toxic_set = set(df_clean['term'].str.lower().str.strip())

        print(f"✅ Tải thành công {len(toxic_set)} từ độc hại từ Blacklist.")
        return toxic_set
    except FileNotFoundError:
        print(f"❌ LỖI: Không tìm thấy file Blacklist tại '{path}'.")
        print("Sử dụng một danh sách rỗng.")
        return set()
    except Exception as e:
        print(f"❌ LỖI khi đọc Blacklist: {e}")
        return set()

# Tải blacklist và lưu vào biến toàn cục
TOXIC_BLACKLIST_SET = load_blacklist(BLACKLIST_PATH)

✅ Tải thành công 39 từ độc hại từ Blacklist.


In [20]:
# --- HÀM PIPELINE TỔNG THỂ (PHIÊN BẢN CẬP NHẬT VỚI BLACKLIST) ---

def check_text_against_blacklist(text, blacklist_set):
    """Kiểm tra xem văn bản có chứa bất kỳ từ nào trong blacklist không."""
    if not text:
        return False

    text_lower = text.lower()
    for word in blacklist_set:
        # Kiểm tra "word" có nằm "trong" text_lower không
        if word in text_lower:
            print(f"   (Phát hiện Blacklist: '{word}')")
            return True
    return False

def multimodal_toxicity_pipeline(comment_text, audio_path, tokenizer_phobert, model_phobert, processor_asr, model_asr, blacklist_set):
    """
    Xây dựng Pipeline đa phương thức (VỚI BLACKLIST).
    """
    print("\n--- BẮT ĐẦU PHÂN TÍCH ĐA PHƯƠNG THỨC ---")

    # 1. PHÂN TÍCH KÊNH VĂN BẢN
    is_toxic_from_blacklist_text = check_text_against_blacklist(comment_text, blacklist_set)
    if is_toxic_from_blacklist_text:
        text_is_toxic, text_conf = 1, 1.0 # Ép kết quả
    else:
        text_is_toxic, text_conf = classify_text_toxicity(comment_text, tokenizer_phobert, model_phobert)

    print(f"Kênh Văn bản ('{comment_text[:30]}...'): {'ĐỘC HẠI' if text_is_toxic else 'SẠCH'} (Conf: {text_conf:.4f})")

    # 2. CHUYỂN ĐỔI GIỌNG NÓI (ASR)
    transcription = transcribe_audio_vietnamese(audio_path, processor_asr, model_asr)

    if transcription is None:
        transcription = ""

    print(f"Bản chép lời (ASR): '{transcription[:50]}...'")

    # 3. PHÂN TÍCH KÊNH ÂM THANH
    is_toxic_from_blacklist_audio = check_text_against_blacklist(transcription, blacklist_set)

    if is_toxic_from_blacklist_audio:
        audio_is_toxic, audio_conf = 1, 1.0 # Ép kết quả
    else:
        audio_is_toxic, audio_conf = classify_text_toxicity(transcription, tokenizer_phobert, model_phobert)

    print(f"Kênh Âm thanh: {'ĐỘC HẠI' if audio_is_toxic else 'SẠCH'} (Conf: {audio_conf:.4f})")

    # 4. QUYẾT ĐỊNH CUỐI CÙNG
    if text_is_toxic == 1 and audio_is_toxic == 1:
        final_decision = "RẤT ĐỘC HẠI (Text + Audio)"
        final_confidence = (text_conf + audio_conf) / 2
    elif text_is_toxic == 1 or audio_is_toxic == 1:
        final_decision = "ĐỘC HẠI (Một kênh)"
        final_confidence = max(text_conf, audio_conf)
    else:
        final_decision = "SẠCH"
        final_confidence = max(text_conf, audio_conf)

    print("\n📢 KẾT QUẢ CUỐI CÙNG:")
    print(f"   Quyết định: {final_decision}")
    print(f"   Độ tin cậy tổng hợp: {final_confidence:.4f}")

    return final_decision, final_confidence

print("✅ Đã định nghĩa các hàm Pipeline MỚI (với Blacklist).")

✅ Đã định nghĩa các hàm Pipeline MỚI (với Blacklist).


# Wave2Vec

In [21]:
!pip install librosa soundfile

In [23]:
import torch
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# --- 1. TẢI LẠI MÔ HÌNH PHOBERT ĐÃ HUẤN LUYỆN ---

MODEL_SAVE_PATH = "./phobert_toxic_weighted_final_model" # Đường dẫn đã lưu

try:
    final_tokenizer = AutoTokenizer.from_pretrained(MODEL_SAVE_PATH)
    final_model = AutoModelForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    final_model.to(device)
    final_model.eval() # Chuyển sang chế độ suy luận

    print(f"✅ Tải thành công mô hình PhoBERT đã huấn luyện từ: {MODEL_SAVE_PATH}")
except Exception as e:
    print(f"❌ LỖI: Không thể tải mô hình từ {MODEL_SAVE_PATH}.")
    print("Vui lòng đảm bảo bạn đã chạy xong bước huấn luyện và mô hình đã được lưu.")
    raise e

✅ Tải thành công mô hình PhoBERT đã huấn luyện từ: ./phobert_toxic_weighted_final_model


In [24]:
# ===================================================================
# SECTION WAV2VEC (ASR - Chuyển đổi Giọng nói thành Văn bản)
# ===================================================================

import torch
import librosa
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import warnings

warnings.filterwarnings("ignore")

print("\n--- [Bước 1] Khởi tạo mô hình ASR (Voice-to-Text) ---")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# Model ID đã sửa lỗi
MODEL_ID = "nguyenvulebinh/wav2vec2-base-vietnamese-250h"

try:
    processor_asr = Wav2Vec2Processor.from_pretrained(MODEL_ID)
    model_asr = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

    model_asr.to(device)
    model_asr.eval()

    print(f"✅ Tải thành công model ASR: {MODEL_ID}")

except Exception as e:
    print(f"❌ Lỗi khi tải model ASR: {e}")
    raise e

# --- Định nghĩa hàm chuyển đổi audio thành text ---
def transcribe_audio_vietnamese(audio_path, processor, model):
    if not audio_path:
        return ""

    try:
        speech_array, sampling_rate = librosa.load(audio_path, sr=16000)
    except Exception as e:
        print(f"❌ Lỗi: Không thể đọc file âm thanh tại '{audio_path}'. Lỗi: {e}")
        return None

    input_values = processor(speech_array, sampling_rate=16000, return_tensors="pt").input_values
    input_values = input_values.to(device)

    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    return transcription.lower()

print("\nĐã định nghĩa hàm: transcribe_audio_vietnamese(audio_path, processor_asr, model_asr)")
print("--- ✅ Hoàn tất chuẩn bị cho Bước 1 ---")


--- [Bước 1] Khởi tạo mô hình ASR (Voice-to-Text) ---
Sử dụng thiết bị: cuda


preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

✅ Tải thành công model ASR: nguyenvulebinh/wav2vec2-base-vietnamese-250h

Đã định nghĩa hàm: transcribe_audio_vietnamese(audio_path, processor_asr, model_asr)
--- ✅ Hoàn tất chuẩn bị cho Bước 1 ---


In [ ]:
# --- ĐỊNH NGHĨA HÀM PHÂN LOẠI VĂN BẢN (CHO PHOBERT) ---
def classify_text_toxicity(text, tokenizer, model):
    """Phân loại văn bản thành Độc hại (1) hoặc Sạch (0) sử dụng PhoBERT."""
    if not text:
        return 0, 0.5 # Nếu văn bản rỗng, coi là sạch

    # Tiền xử lý văn bản (Làm sạch)
    text_clean = re.sub(r'[^\w\s]', ' ', str(text).lower()).strip()

    # Tokenize
    inputs = tokenizer(text_clean, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    # Suy luận
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    # Lấy xác suất và nhãn dự đoán
    probabilities = torch.softmax(outputs.logits, dim=1)[0].tolist()
    predicted_class_id = torch.argmax(outputs.logits, dim=1).item()

    is_toxic = predicted_class_id
    confidence = probabilities[predicted_class_id]

    return is_toxic, confidence

print("✅ Đã định nghĩa hàm: classify_text_toxicity")

✅ Đã định nghĩa hàm: classify_text_toxicity


# Test

In [ ]:
import os
import re
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# --- 1. TẢI LẠI MÔ HÌNH PHOBERT (Đảm bảo đã chạy khối tải mô hình) ---
# Đường dẫn nơi bạn đã lưu mô hình
MODEL_SAVE_PATH = "./phobert_toxic_weighted_final_model"

try:
    # Tải Tokenizer và Mô hình PhoBERT đã tinh chỉnh
    final_tokenizer = AutoTokenizer.from_pretrained(MODEL_SAVE_PATH)
    final_model = AutoModelForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)
    final_model.eval() # Chuyển sang chế độ suy luận
    print("✅ Mô hình PhoBERT đã tải lại thành công.")
except Exception as e:
    print(f"❌ LỖI: Không thể tải mô hình từ {MODEL_SAVE_PATH}. Vui lòng đảm bảo bạn đã chạy xong bước huấn luyện và lưu mô hình.")
    print(f"Chi tiết lỗi: {e}")
    # Dừng nếu lỗi tải
    raise

# --- 2. ĐỊNH NGHĨA HÀM PHÂN LOẠI VĂN BẢN (Sao chép lại để tiện kiểm tra) ---
def classify_text_toxicity(text, tokenizer, model):
    """Phân loại văn bản thành Độc hại (1) hoặc Sạch (0) sử dụng PhoBERT."""
    if not text:
        return 0, 0.5 # Nếu văn bản rỗng, coi là sạch

    # Tiền xử lý văn bản (Làm sạch tương tự như bước huấn luyện)
    # Lưu ý: Hàm này cần khớp với hàm basic_cleanup trong quá trình huấn luyện
    text_clean = re.sub(r'[^\w\s]', ' ', str(text).lower()).strip()

    # Tokenize
    inputs = tokenizer(text_clean, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    # Suy luận
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    # Lấy xác suất và nhãn dự đoán
    probabilities = torch.softmax(outputs.logits, dim=1)[0].tolist()
    predicted_class_id = torch.argmax(outputs.logits, dim=1).item()

    # Trả về nhãn (0 hoặc 1) và độ tin cậy của nhãn đó
    is_toxic = predicted_class_id
    confidence = probabilities[predicted_class_id]

    return is_toxic, confidence

# --- 3. ĐỊNH NGHĨA CÁC VÍ DỤ KIỂM THỬ ---
test_cases = [
    ("Phân biệt đối xử là hành vi không thể chấp nhận được.", "Sạch (Clean)"),
    ("Bạn làm tốt lắm, mọi người đều hài lòng về điều đó.", "Sạch (Clean)"),
    ("Mày đúng là đồ ngu dốt, cút ngay đi!", "Độc hại (Toxic)"),
    ("Tại sao lại có người phát ngôn những lời ngu ngốc như vậy?", "Độc hại (Toxic)"),
    ("Việc học hành của học sinh nên được quan tâm nhiều hơn.", "Sạch (Clean)"),
    ("Con chó nhà mày bị khùng à?", "Độc hại (Toxic)"),
]

print("\n--- KẾT QUẢ KIỂM THỬ MÔ HÌNH PHOBERT ---")
for text, expected in test_cases:
    # Phân loại
    is_toxic, confidence = classify_text_toxicity(text, final_tokenizer, final_model)

    # Định dạng kết quả
    result = "ĐỘC HẠI (Toxic)" if is_toxic == 1 else "SẠCH (Clean)"

    # In ra kết quả
    print("-" * 50)
    print(f"Văn bản: '{text}'")
    print(f"Dự đoán: {result} | Độ tin cậy: {confidence:.4f}")
    print(f"Kỳ vọng: {expected}")

✅ Mô hình PhoBERT đã tải lại thành công.

--- KẾT QUẢ KIỂM THỬ MÔ HÌNH PHOBERT ---


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

--------------------------------------------------
Văn bản: 'Phân biệt đối xử là hành vi không thể chấp nhận được.'
Dự đoán: SẠCH (Clean) | Độ tin cậy: 0.9741
Kỳ vọng: Sạch (Clean)
--------------------------------------------------
Văn bản: 'Bạn làm tốt lắm, mọi người đều hài lòng về điều đó.'
Dự đoán: SẠCH (Clean) | Độ tin cậy: 0.9941
Kỳ vọng: Sạch (Clean)
--------------------------------------------------
Văn bản: 'Mày đúng là đồ ngu dốt, cút ngay đi!'
Dự đoán: ĐỘC HẠI (Toxic) | Độ tin cậy: 0.9597
Kỳ vọng: Độc hại (Toxic)
--------------------------------------------------
Văn bản: 'Tại sao lại có người phát ngôn những lời ngu ngốc như vậy?'
Dự đoán: ĐỘC HẠI (Toxic) | Độ tin cậy: 0.9420
Kỳ vọng: Độc hại (Toxic)
--------------------------------------------------
Văn bản: 'Việc học hành của học sinh nên được quan tâm nhiều hơn.'
Dự đoán: SẠCH (Clean) | Độ tin cậy: 0.9889
Kỳ vọng: Sạch (Clean)
--------------------------------------------------
Văn bản: 'Con chó nhà mày bị khùng à?'
Dự đo

In [ ]:
# --- 4. KIỂM THỬ PIPELINE ĐA PHƯƠNG THỨC (CẬP NHẬT) ---

# Giữ nguyên file test đã bị lỗi
path_to_your_audio_file = "/content/drive/My Drive/DỰ ÁN CNTT/ViCTSD/conmemay_1.aac"
example_comment = "Video này hay quá"

print(f"--- Bắt đầu kiểm thử Pipeline tổng thể (với Blacklist) ---")
print(f"File âm thanh: {path_to_your_audio_file}")
print(f"Bình luận kèm theo: {example_comment}")

try:
    multimodal_toxicity_pipeline(
        comment_text=example_comment,
        audio_path=path_to_your_audio_file,
        tokenizer_phobert=final_tokenizer,
        model_phobert=final_model,
        processor_asr=processor_asr,
        model_asr=model_asr,
        blacklist_set=TOXIC_BLACKLIST_SET # <<< THÊM THAM SỐ QUAN TRỌNG NÀY
    )
except NameError as e:
    if 'TOXIC_BLACKLIST_SET' in str(e):
        print("\n❌ LỖI: Bạn chưa chạy ô code 'TẢI BLACKLIST' (Bước 1).")
    else:
        print(f"\n❌ LỖI: Biến không tồn tại: {e}")
except Exception as e:
    print(f"\n❌ LỖI BẤT NGỜ: {e}")

--- Bắt đầu kiểm thử Pipeline tổng thể (với Blacklist) ---
File âm thanh: /content/drive/My Drive/DỰ ÁN CNTT/ViCTSD/conmemay_1.aac
Bình luận kèm theo: Video này hay quá

--- BẮT ĐẦU PHÂN TÍCH ĐA PHƯƠNG THỨC ---
Kênh Văn bản ('Video này hay quá...'): SẠCH (Conf: 0.9938)
Bản chép lời (ASR): 'con mẹ mày con mẹ may con mẹ may thăn chá...'
   (Phát hiện Blacklist: 'con mẹ')
Kênh Âm thanh: ĐỘC HẠI (Conf: 1.0000)

📢 KẾT QUẢ CUỐI CÙNG:
   Quyết định: ĐỘC HẠI (Một kênh)
   Độ tin cậy tổng hợp: 1.0000


# Giao diện

In [25]:
!ls ./phobert_toxic_weighted_final_model

added_tokens.json  model.safetensors	    training_args.bin
bpe.codes	   special_tokens_map.json  vocab.txt
config.json	   tokenizer_config.json


In [ ]:
!pip install gradio
!pip install gradio soundfile

# 08/11

In [ ]:
# ===================================================================
# SECTION GRADIO (TẢI MODEL, XỬ LÝ, VÀ CHẠY GIAO DIỆN)
# ===================================================================

# --- 0. IMPORT CÁC THƯ VIỆN CẦN THIẾT ---
import torch
import re
import librosa
import pandas as pd
import easyocr
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Wav2Vec2Processor, Wav2Vec2ForCTC
import warnings

warnings.filterwarnings("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Sử dụng thiết bị: {device}")

# --- 1A. TẢI CÁC MÔ HÌNH AI (PHOBERT, WAV2VEC, EASYOCR) ---

MODEL_SAVE_PATH = "./phobert_toxic_weighted_final_model"
ASR_MODEL_ID = "nguyenvulebinh/wav2vec2-base-vietnamese-250h"

try:
    print("--- Đang tải mô hình PhoBERT... ---")
    final_tokenizer = AutoTokenizer.from_pretrained(MODEL_SAVE_PATH)
    final_model = AutoModelForSequenceClassification.from_pretrained(MODEL_SAVE_PATH).to(device).eval()

    print("--- Đang tải mô hình Wav2Vec (ASR)... ---")
    processor_asr = Wav2Vec2Processor.from_pretrained(ASR_MODEL_ID)
    model_asr = Wav2Vec2ForCTC.from_pretrained(ASR_MODEL_ID).to(device).eval()

    print("--- Đang tải mô hình EasyOCR... ---")
    reader = easyocr.Reader(['vi', 'en'], gpu=torch.cuda.is_available())

    print("✅ Tải thành công tất cả 3 mô hình (PhoBERT, Wav2Vec, EasyOCR).")

except Exception as e:
    print(f"❌ LỖI TẢI MÔ HÌNH: {e}")
    print("Vui lòng đảm bảo đã chạy thành công khối huấn luyện và lưu mô hình.")

# --- 1B. TẢI BLACKLIST VÀ CÁC HÀM HỖ TRỢ ---

BLACKLIST_PATH = "/content/drive/My Drive/DỰ ÁN CNTT/ViCTSD/blacklist.csv"

def load_blacklist(path):
    try:
        df = pd.read_csv(path)
        df_clean = df[~df['term'].str.startswith('#', na=False)]
        toxic_set = set(df_clean['term'].str.lower().str.strip())
        print(f"✅ Tải thành công {len(toxic_set)} từ độc hại từ Blacklist.")
        return toxic_set
    except Exception as e:
        print(f"❌ LỖI khi đọc Blacklist: {e}")
        return set()

TOXIC_BLACKLIST_SET = load_blacklist(BLACKLIST_PATH)

def check_text_against_blacklist(text, blacklist_set):
    """Kiểm tra text với blacklist. Trả về (Bool, Word)."""
    if not text:
        return False, None
    text_lower = str(text).lower()
    for word in blacklist_set:
        if word in text_lower:
            return True, word # Trả về True và từ bị phát hiện
    return False, None

# --- 1C. ĐỊNH NGHĨA CÁC HÀM CORE (Từ Khối 1 cũ) ---

def basic_cleanup(text):
    text = str(text)
    # Sửa lỗi regex: Bỏ bớt 1 dấu \
    text = re.sub(r'[^\w\s]', ' ', text).lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def classify_text_toxicity(text, tokenizer, model):
    if not text:
        return 0, 0.5
    text_clean = basic_cleanup(text)
    inputs = tokenizer(text_clean, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    probabilities = torch.softmax(outputs.logits, dim=1)[0].tolist()
    predicted_class_id = torch.argmax(outputs.logits, dim=1).item()
    return predicted_class_id, probabilities[predicted_class_id]

def transcribe_audio(audio_path, processor, model):
    if not audio_path:
        return ""
    try:
        speech_array, _ = librosa.load(audio_path, sr=16000)
    except Exception as e:
        return f"Lỗi đọc file âm thanh: {e}"

    input_values = processor(speech_array, sampling_rate=16000, return_tensors="pt").input_values.to(device)
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # Sửa lỗi: Dùng processor.batch_decode thay vì .decode
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription.lower()

# --- 1D. ĐỊNH NGHĨA CÁC HÀM HIGHLIGHT & OCR (Từ code cũ) ---
# (Lấy từ các ô code trước đó của bạn)
ALL_WORDS_TO_HIGHLIGHT = TOXIC_BLACKLIST_SET.union(set([
    'ngu dốt', 'cút', 'chó', 'điên', 'óc chó', 'rác rưởi', 'khùng',
    'mày', 'tao', 'đồ ngu', 'mất dạy', 'chết đi', 'thằng', 'con'
]))

def highlight_toxic_words(text):
    if not text: return ""
    highlighted_text = str(text)
    sorted_words = sorted(list(ALL_WORDS_TO_HIGHLIGHT), key=len, reverse=True)
    for word in sorted_words:
        pattern = r'\b(' + re.escape(word) + r')\b'
        highlighted_text = re.sub(
            pattern,
            r'<span style="color:red; font-weight:bold;">\1</span>',
            highlighted_text,
            flags=re.IGNORECASE
        )
    return highlighted_text

def ocr_and_predict(image_file, tokenizer, model, reader):
    ocr_text = ""
    is_toxic = 0
    confidence = 0.5
    if image_file is not None and reader is not None:
        try:
            results = reader.readtext(image_file) # image_file là đường dẫn
            if results:
                ocr_text = " ".join([res[1] for res in results])
            if ocr_text:
                is_toxic, confidence = classify_text_toxicity(ocr_text, tokenizer, model)
        except Exception as e:
            ocr_text = f"❌ LỖI OCR: {e}"
    return ocr_text, is_toxic, confidence

# --- 2. HÀM PIPELINE (THAY THẾ Khối 2 CŨ) ---
import soundfile as sf
import os

# --- THAY THẾ TOÀN BỘ HÀM NÀY ---
def gradio_multimodal_pipeline(comment_text, audio_input_numpy, image_file):

    # --- 1. KÊNH VĂN BẢN (Text/Comment) ---
    # (Giữ nguyên, không thay đổi)
    reason_text = ""
    is_blacklisted_text, word_text = check_text_against_blacklist(comment_text, TOXIC_BLACKLIST_SET)
    if is_blacklisted_text:
        text_is_toxic, text_conf = 1, 1.0
        reason_text = f"(Blacklist: '{word_text}')"
    else:
        text_is_toxic, text_conf = classify_text_toxicity(comment_text, final_tokenizer, final_model)
    text_highlighted = highlight_toxic_words(comment_text)

    # --- 2. KÊNH ÂM THANH (ASR) ---
    # (Đây là phần được cập nhật)
    transcription, audio_is_toxic, audio_conf, reason_audio = "", 0, 0.5, ""

    # Tên file tạm
    temp_audio_path = "temp_recorded_audio.wav"
    audio_path_to_process = None

    # Xử lý input (có thể là file upload hoặc ghi âm)
    if audio_input_numpy is not None:
        if isinstance(audio_input_numpy, str):
            # Trường hợp 1: Người dùng UPLOAD file (Gradio trả về filepath)
            audio_path_to_process = audio_input_numpy
        else:
            # Trường hợp 2: Người dùng GHI ÂM (Gradio trả về (rate, numpy_array))
            try:
                sampling_rate, audio_data = audio_input_numpy
                sf.write(temp_audio_path, audio_data, sampling_rate)
                audio_path_to_process = temp_audio_path
            except Exception as e:
                print(f"Lỗi lưu file âm thanh tạm: {e}")

    # BÂY GIỜ MỚI XỬ LÝ FILE (ĐÃ ĐẢM BẢO LÀ FILEPATH)
    if audio_path_to_process is not None:
        transcription = transcribe_audio(audio_path_to_process, processor_asr, model_asr)
        is_blacklisted_audio, word_audio = check_text_against_blacklist(transcription, TOXIC_BLACKLIST_SET)

        if is_blacklisted_audio:
            audio_is_toxic, audio_conf = 1, 1.0
            reason_audio = f"(Blacklist: '{word_audio}')"
        else:
            audio_is_toxic, audio_conf = classify_text_toxicity(transcription, final_tokenizer, final_model)

    audio_highlighted = highlight_toxic_words(transcription)

    # (Tùy chọn) Xóa file tạm
    if os.path.exists(temp_audio_path):
        os.remove(temp_audio_path)

    # --- 3. KÊNH HÌNH ẢNH (OCR) ---
    # (Giữ nguyên, không thay đổi)
    ocr_text, image_is_toxic, image_conf, reason_image = "", 0, 0.5, ""
    image_output = "<b>Không có Hình ảnh.</b>"
    if image_file is not None and reader is not None:
        ocr_text, ocr_is_toxic_model, ocr_conf_model = ocr_and_predict(
            image_file, final_tokenizer, final_model, reader
        )
        is_blacklisted_ocr, word_ocr = check_text_against_blacklist(ocr_text, TOXIC_BLACKLIST_SET)
        if is_blacklisted_ocr:
            image_is_toxic, image_conf = 1, 1.0
            reason_image = f"(Blacklist: '{word_ocr}')"
        else:
            image_is_toxic, image_conf = ocr_is_toxic_model, ocr_conf_model
    ocr_highlighted = highlight_toxic_words(ocr_text)

    # --- 4. TỔNG HỢP QUYẾT ĐỊNH ---
    # (Giữ nguyên, không thay đổi)
    is_toxic = (text_is_toxic == 1) or (audio_is_toxic == 1) or (image_is_toxic == 1)
    if is_toxic:
        final_decision_str = "RẤT ĐỘC HẠI" if (text_is_toxic + audio_is_toxic + image_is_toxic) > 1 else "ĐỘC HẠI"
        final_color = "red" if (text_is_toxic + audio_is_toxic + image_is_toxic) > 1 else "orange"
        toxic_confs = [c for t, c in zip([text_is_toxic, audio_is_toxic, image_is_toxic], [text_conf, audio_conf, image_conf]) if t == 1]
        final_confidence = max(toxic_confs) if toxic_confs else 0.5
    else:
        final_decision_str = "SẠCH"
        final_color = "green"
        final_confidence = max(text_conf, audio_conf, image_conf)

    # --- 5. ĐỊNH DẠNG OUTPUT HTML ---
    # (Giữ nguyên, không thay đổi)
    text_status = "ĐỘC HẠI" if text_is_toxic else "SẠCH"
    text_color = "red" if text_is_toxic else "green"
    text_output = f"<b>Bình luận (Text):</b> {text_highlighted}<br>"
    text_output += f"<b>Phân loại Text:</b> <span style='color:{text_color}'>{text_status}</span> (Conf: {text_conf:.4f}) <i>{reason_text}</i>"

    if transcription and "Lỗi" not in transcription:
        asr_status = "ĐỘC HẠI" if audio_is_toxic else "SẠCH"
        asr_color = "red" if audio_is_toxic else "green"
        audio_output = f"<b>Bản chép lời (ASR):</b> <i>'{audio_highlighted}'</i><br>"
        audio_output += f"<b>Phân loại Audio:</b> <span style='color:{asr_color}'>{asr_status}</span> (Conf: {audio_conf:.4f}) <i>{reason_audio}</i>"
    elif audio_input_numpy is not None:
        audio_output = "<b>Không nhận dạng được giọng nói (hoặc nói quá nhỏ).</b>"
    else:
        audio_output = "<b>Không có Audio.</b>"

    if image_file is not None:
        image_status = "ĐỘC HẠI" if image_is_toxic else "SẠCH"
        image_color = "red" if image_is_toxic else "green"
        if "LỖI OCR" not in ocr_text:
            image_output = f"<b>Văn bản OCR:</b> {ocr_highlighted}<br>"
            image_output += f"<b>Phân loại OCR:</b> <span style='color:{image_color}'>{image_status}</span> (Conf: {image_conf:.4f}) <i>{reason_image}</i>"
        else:
            image_output = f"<b>Trạng thái OCR:</b> <span style='color:gray'>{ocr_text}</span>"

    final_output = f"<h2 style='color:{final_color};'>📢 KẾT QUẢ CUỐI CÙNG: {final_decision_str}</h2>"
    final_output += f"<h3>Độ tin cậy tổng hợp: {final_confidence:.4f}</h3>"

    return final_output, text_output, audio_output, image_output

# --- 3. KHỞI TẠO GIAO DIỆN (GIỮ NGUYÊN Khối 3 CŨ) ---
print("\n--- 3. Khởi tạo Giao diện Gradio ---")
inputs = [
    gr.Textbox(label="1. Nhập Bình luận (Văn bản)", placeholder="Nhập bình luận hoặc phụ đề video vào đây..."),
    gr.Audio(label="2. Tải lên hoặc Ghi âm Giọng nói", type="filepath", sources=["upload", "microphone"]),
    gr.Image(label="3. Tải lên Hình ảnh/Phụ đề (OCR)", type="filepath", sources=["upload"], height=150)
]
outputs = [
    gr.HTML(label="Quyết định Chung"),
    gr.HTML(label="Phân tích Kênh Văn bản (Đã tô màu)"),
    gr.HTML(label="Phân tích Kênh Âm thanh (ASR)"),
    gr.HTML(label="Phân tích Kênh Hình ảnh (OCR)")
]
iface = gr.Interface(
    fn=gradio_multimodal_pipeline, # Gọi hàm mới đã cập nhật
    inputs=inputs,
    outputs=outputs,
    title="HỆ THỐNG NHẬN DẠNG TỪ NGỮ ĐỘC HẠI ĐA PHƯƠNG THỨC (PhoBERT + Wav2Vec)",
    description="Kiểm tra mức độ Độc hại (Toxic) dựa trên Văn bản, Giọng nói (ASR) và Hình ảnh (OCR). Hệ thống sử dụng AI (PhoBERT) và Blacklist.",
    examples=[
        ["Mày đúng là đồ ngu dốt, cút ngay đi!", None, None],
        ["con mẹ mày", None, None], # Test Blacklist
        ["Việc học hành của học sinh nên được quan tâm nhiều hơn.", None, None],
        # Thêm ví dụ file audio (phải có file thật trên server Colab)
        ["Comment sạch", "/content/drive/My Drive/DỰ ÁN CNTT/ViCTSD/conmemay_1.aac", None],
    ]
)

# Chạy giao diện
print("--- Đang khởi chạy Giao diện Gradio... ---")
iface.launch(share=True, debug=True)

Sử dụng thiết bị: cuda
--- Đang tải mô hình PhoBERT... ---
--- Đang tải mô hình Wav2Vec (ASR)... ---


--- Đang tải mô hình EasyOCR... ---
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete✅ Tải thành công tất cả 3 mô hình (PhoBERT, Wav2Vec, EasyOCR).
✅ Tải thành công 39 từ độc hại từ Blacklist.

--- 3. Khởi tạo Giao diện Gradio ---
--- Đang khởi chạy Giao diện Gradio... ---
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fca5de56d83933f746.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#

# Old

In [ ]:
# --- Tải lại Mô hình PhoBERT (Đảm bảo đã chạy khối tải mô hình) ---
MODEL_SAVE_PATH = "phobert_toxic_weighted_final_model"
ASR_MODEL_ID = "nguyenvulebinh/wav2vec2-base-vietnamese-250h"

try:
    # Tải PhoBERT và Wav2Vec 2.0... (giữ nguyên)

    # KHỞI TẠO EASYOCR READER
    reader = easyocr.Reader(['vi', 'en'], gpu=torch.cuda.is_available()) # Sử dụng GPU nếu có
    print("✅ EasyOCR Reader đã được tải thành công.")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    final_model.to(device)
    model_asr.to(device)
    print("Các mô hình đã được tải thành công và sẵn sàng.")

except Exception as e:
    print(f"❌ LỖI TẢI MÔ HÌNH: {e}")
    print("Vui lòng đảm bảo đã chạy thành công khối huấn luyện và lưu mô hình.")

# --- Định nghĩa lại các hàm core (Sao chép từ file gốc) ---

def basic_cleanup(text):
    text = str(text)
    text = re.sub(r'[^\\w\\s]', ' ', text).lower()
    text = re.sub(r'\\s+', ' ', text).strip()
    return text

def classify_text_toxicity(text, tokenizer, model):
    if not text:
        return 0, 0.5

    text_clean = basic_cleanup(text)
    inputs = tokenizer(text_clean, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = torch.softmax(outputs.logits, dim=1)[0].tolist()
    predicted_class_id = torch.argmax(outputs.logits, dim=1).item()

    is_toxic = predicted_class_id
    confidence = probabilities[predicted_class_id]

    return is_toxic, confidence

def transcribe_audio(audio_path, processor, model):
    if not audio_path:
        return ""
    try:
        # Đọc file âm thanh và chuẩn hóa (Wav2Vec 2.0 yêu cầu 16kHz)
        speech_array, _ = librosa.load(audio_path, sr=16000)
    except Exception as e:
        return f"Lỗi đọc file âm thanh: {e}"

    input_values = processor(speech_array, sampling_rate=16000, return_tensors="pt").input_values.to(device)

    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])

    return transcription.lower()

✅ EasyOCR Reader đã được tải thành công.
Các mô hình đã được tải thành công và sẵn sàng.


In [ ]:
# CẬP NHẬT: Thêm image_file vào đầu vào của pipeline
def gradio_multimodal_pipeline(comment_text, audio_file, image_file):
    """
    Hàm xử lý đa phương thức (Text + ASR + OCR) cho Gradio.
    """
    # 1. PHÂN TÍCH KÊNH VĂN BẢN (Text/Comment)
    text_is_toxic, text_conf = classify_text_toxicity(comment_text, final_tokenizer, final_model)
    text_highlighted = highlight_toxic_words(comment_text) # Tô màu cho văn bản gốc

    # 2. XỬ LÝ KÊNH ÂM THANH (ASR)
    transcription = ""
    audio_is_toxic = 0
    audio_conf = 0.5

    if audio_file is not None:
        transcription = transcribe_audio(audio_file, processor_asr, model_asr)
        # 3. PHÂN TÍCH VĂN BẢN ASR
        audio_is_toxic, audio_conf = classify_text_toxicity(transcription, final_tokenizer, final_model)

    audio_highlighted = highlight_toxic_words(transcription) # Tô màu cho bản chép lời

    # 4. XỬ LÝ KÊNH HÌNH ẢNH (OCR)
    ocr_text = ""
    image_is_toxic = 0
    image_conf = 0.5
    image_output = "<b>Không có Hình ảnh.</b>" # Mặc định

    if image_file is not None and reader is not None:
        ocr_text, image_is_toxic, image_conf = ocr_and_predict(
            image_file, final_tokenizer, final_model, reader
        )

    ocr_highlighted = highlight_toxic_words(ocr_text) # Tô màu cho văn bản OCR

    # 5. TỔNG HỢP QUYẾT ĐỊNH CUỐI CÙNG (Logic OR với 3 kênh)
    is_toxic = (text_is_toxic == 1) or (audio_is_toxic == 1) or (image_is_toxic == 1)

    # Tính toán độ tin cậy và quyết định cuối cùng
    if is_toxic:
        final_decision_str = "RẤT ĐỘC HẠI" if (text_is_toxic + audio_is_toxic + image_is_toxic) > 1 else "ĐỘC HẠI"
        final_color = "red" if (text_is_toxic + audio_is_toxic + image_is_toxic) > 1 else "orange"

        # Lấy độ tin cậy tối đa từ các kênh báo toxic
        toxic_confs = [c for t, c in zip([text_is_toxic, audio_is_toxic, image_is_toxic], [text_conf, audio_conf, image_conf]) if t == 1]
        final_confidence = max(toxic_confs) if toxic_confs else 0.5
    else:
        final_decision_str = "SẠCH"
        final_color = "green"
        final_confidence = max(text_conf, audio_conf, image_conf) # Lấy độ tin cậy cao nhất cho nhãn sạch

    # --- ĐỊNH DẠNG KẾT QUẢ ĐỂ HIỂN THỊ TRONG GRADIO ---

    # Kênh Văn bản
    text_status = "ĐỘC HẠI" if text_is_toxic else "SẠCH"
    text_color = "red" if text_is_toxic else "green"
    text_output = f"<b>Bình luận (Text):</b> {text_highlighted}<br>"
    text_output += f"<b>Phân loại Text:</b> <span style='color:{text_color}'>{text_status}</span> (Conf: {text_conf:.4f})"

    # Kênh Âm thanh
    if transcription and transcription != "":
        asr_status = "ĐỘC HẠI" if audio_is_toxic else "SẠCH"
        asr_color = "red" if audio_is_toxic else "green"
        audio_output = f"<b>Bản chép lời (ASR):</b> <i>'{audio_highlighted}'</i><br>"
        audio_output += f"<b>Phân loại Audio:</b> <span style='color:{asr_color}'>{asr_status}</span> (Conf: {audio_conf:.4f})"
    else:
        audio_output = "<b>Không có Audio.</b>"

    # Kênh Hình ảnh (OCR)
    if image_file is not None:
        image_status = "ĐỘC HẠI" if image_is_toxic else "SẠCH"
        image_color = "red" if image_is_toxic else "green"

        if "LỖI OCR" not in ocr_text:
            image_output = f"<b>Văn bản OCR:</b> {ocr_highlighted}<br>"
            image_output += f"<b>Phân loại OCR:</b> <span style='color:{image_color}'>{image_status}</span> (Conf: {image_conf:.4f})"
        else:
             image_output = f"<b>Trạng thái OCR:</b> <span style='color:gray'>{ocr_text}</span>" # Hiển thị lỗi

    # Quyết định cuối cùng
    final_output = f"<h2 style='color:{final_color};'>📢 KẾT QUẢ CUỐI CÙNG: {final_decision_str}</h2>"
    final_output += f"<h3>Độ tin cậy tổng hợp: {final_confidence:.4f}</h3>"

    # Trả về các output đã cập nhật
    return final_output, text_output, audio_output, image_output

In [ ]:
# --- 3. KHỞI TẠO GIAO DIỆN GRADIO (CẬP NHẬT) ---

# Định nghĩa các thành phần nhập và xuất (THÊM HÌNH ẢNH)
inputs = [
    gr.Textbox(label="1. Nhập Bình luận (Văn bản)", placeholder="Nhập bình luận hoặc phụ đề video vào đây..."),
    gr.Audio(label="2. Tải lên hoặc Ghi âm Giọng nói", type="numpy", sources=["upload", "microphone"]),
    gr.Image(label="3. Tải lên Hình ảnh/Phụ đề (OCR)", type="filepath", sources=["upload"], height=150) # Input mới
]

outputs = [
    gr.HTML(label="Quyết định Chung"),
    gr.HTML(label="Phân tích Kênh Văn bản (Đã tô màu)"),
    gr.HTML(label="Phân tích Kênh Âm thanh (ASR)"),
    gr.HTML(label="Phân tích Kênh Hình ảnh (OCR)") # Output mới
]

# Tạo giao diện
iface = gr.Interface(
    fn=gradio_multimodal_pipeline,
    inputs=inputs,
    outputs=outputs,
    title="HỆ THỐNG NHẬN DẠNG TỪ NGỮ ĐỘC HẠI ĐA PHƯƠNG THỨC (PhoBERT + Wav2Vec 2.0 + OCR)",
    description="Kiểm tra mức độ Độc hại (Toxic) dựa trên Văn bản, Giọng nói (ASR) và Hình ảnh (OCR). Từ độc hại sẽ được tô màu đỏ.",
    examples=[
        # Ví dụ Text only
        ["Mày đúng là đồ ngu dốt, cút ngay đi!", None, None],
        # Ví dụ Clean
        ["Việc học hành của học sinh nên được quan tâm nhiều hơn.", None, None],
        # Thêm ví dụ có file ảnh hợp lệ nếu có (ví dụ: [None, None, "path/to/toxic_meme.jpg"])
    ]
)

# Chạy giao diện
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bb865668afc1480d72.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
